# Gaussian Process Regression - Trend Fitting with a RBF Kernel

In [563]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from ipywidgets import interact

rng = np.random.default_rng()

## Noisless data

We fit a GRP to noiless data with a linear/quadratic trend using a single RBF kernel. For larger length scales, the fit better approximates the linear/quadratic trend. This comes at a cost however. To see we can we expand the mean function $m(x)$ as linear combination of kernel functions
$$m(x) = \sum_{i=1}^n \alpha_i k(x,x_i)$$
with $k(x,y)$ the kernel function and $\vec{\alpha} = (K+\sigma_n^2 I)^{-1} \vec{y}$ the weight vector in the dual space. For larger and larger length scales $K(x,y)\rightarrow$ resulting the "weights" $\alpha_i$ to blow up.


In [595]:
def example_lin_gp(length_scale, type ='linear'):
    x_train = np.linspace(-5,5,6)

    if type == 'linear':
        y_train = x_train
    elif type == 'quadratic':
        y_train = x_train**2
    else:
        raise ValueError('Type: {type} not supported')
    
    y_train = y_train - y_train.mean()

    y_range = np.array([y_train.min(), y_train.max()])

    x_pred = np.linspace(-15,15,100)

    X = x_train.reshape(-1,1)
    Xp = x_pred.reshape(-1,1)

    rbf_kernel = RBF(length_scale=length_scale,length_scale_bounds='fixed')
    gp = GaussianProcessRegressor(kernel=rbf_kernel, normalize_y=False, alpha = 1e-6)
    gp.fit(X, y_train)

    y_mean_pred, y_std_pred = gp.predict(Xp, return_std=True)

    K_XpX = gp.kernel_(Xp,X)
    rbfs = (K_XpX.T * gp.alpha_[:,None])

    fig, (ax1, ax2) = plt.subplots(2)
    ax1.scatter(x_train, y_train, label = 'data')
    ax1.plot(x_pred,y_mean_pred, label = 'm(x)')
    
    ax1.set_ylim(2*y_range)
    ax1.legend()
    ax1.set_ylabel('y')
    ax1.legend()

    ax1.fill_between(
        x_pred,
        y_mean_pred - y_std_pred,
        y_mean_pred + y_std_pred,
        color="tab:blue",
        alpha=0.2,
    )

    for rbf in rbfs:
        ax2.plot(x_pred, rbf)
    ax2.set_xlabel('x')

    plt.show()

In [596]:
interact(example_lin_gp, length_scale=(0.5,5.0,0.5), type=['linear','quadratic']);

interactive(children=(FloatSlider(value=2.5, description='length_scale', max=5.0, min=0.5, step=0.5), Dropdown…

### Description:
**Top**:
The fittted mean function $m(x)$, shaded region is the mean plus/minus one standard deviation

**Bottom**:
The decomposition of the mean function in terms of the RBFs

* For length scales < 2 the data is "overfitted", ie it doesn't fit the linear trend
* When increasing the length scale, the fit improves (closer matched the linear/quadratic trend). However as can be seen in the bottom panel the "weight" tend to blow up for larger length scales, making the solution less numerically robust.


## Trend with Noise

In [573]:
x_train = np.linspace(-5,5,24)
y_train = rng.normal(x_train,1.0)
y_train = y_train - y_train.mean()

y_range = np.array([y_train.min(), y_train.max()])

x_pred = np.linspace(-15,15,100)

X = x_train.reshape(-1,1)
Xp = x_pred.reshape(-1,1)

def example_lin_gp_noise(length_scale):

    kernel = 1.0**2 * RBF(length_scale=length_scale,length_scale_bounds='fixed') + WhiteKernel(noise_level=1.0**2)
    gp = GaussianProcessRegressor(kernel=kernel, normalize_y=False)

    gp.fit(X, y_train)

    y_mean_pred, y_cov_pred = gp.predict(Xp, return_cov=True)

    y_std_pred = np.sqrt(np.diag(y_cov_pred))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,4))

    ax1.scatter(x_train,y_train)
    ax1.plot(x_pred, y_mean_pred)
    ax1.set_ylim(1.5*y_range)

    ax1.fill_between(
        x_pred,
        y_mean_pred - y_std_pred,
        y_mean_pred + y_std_pred,
        color="tab:blue",
        alpha=0.2,
    )

    x_coor, y_coor = np.meshgrid(x_pred,x_pred)

    z_range = np.max(np.abs(y_cov_pred))
    ax2.pcolormesh(x_coor,y_coor,-y_cov_pred,cmap='RdBu',vmin=-z_range,vmax=z_range)
    ax2.set
    ax2.set_title("COV(X',X')")

    plt.show()

In [574]:
interact(example_lin_gp_noise,length_scale=(0.5,10.0,0.1));

interactive(children=(FloatSlider(value=5.2, description='length_scale', max=10.0, min=0.5), Output()), _dom_c…

### Description:
**Left**:
The fittted mean function $m(x)$, shaded region is the mean plus/minus one standard deviation

**Right**:
The covariance function $cov(x,x)$